In [137]:
#import important libraries
from nltk import sent_tokenize, word_tokenize
from gensim.models import Word2Vec 
from numpy.linalg import norm
from numpy import dot
import string
import pandas as pd
import numpy as np
import gensim

In [138]:
sample = open('data_preprocessed.txt','r')
s = sample.read()
f = s.replace('\n', ' ')

In [139]:
#prepare data for Word2Vec model
data = []

# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []
    
    # tokenize the setence into words
    for j in word_tokenize(i):
        temp.append(j)
        
    data.append(temp)

In [140]:
# train the model
model = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 3)

In [141]:
#function to get the word frequency of each word in a verse/sentence

# list_of_words: a pre-processed list of words in a verse/sentence.
def word_count(list_of_words):
    
    #dictionary keeping frequency for all the unique words
    counts = dict()
    
    for word in list_of_words:
        #if word already listed, increase its count
        if word in counts:
            counts[word] += 1
        # else putting the new word in dictionary
        else:
            counts[word] = 1

    return counts

In [142]:
#function to calculate composite sentence/verse embeddings
#verse to be given in tokenzied format to func.
def senemb(alpha, verse_tok, model, dim):
    # create an empty vector
    sentEmb = np.zeros(dim)
    v_count = word_count(verse_tok)
    
    for word in verse_tok:
        prob = alpha / (alpha + v_count[word]/len(verse))
        sentEmb = sentEmb +  model.wv[word]  * prob
        
    return sentEmb/len(verse)

In [143]:
#sentence embedding for verses

verses = pd.read_csv('verses.csv', usecols=['#verse', 'Translation'])
data = verses['Translation'].to_list()

#list to store all the vectors
verse_vectors = []

# loop over all the verses
for verse in data:
    # tokenize and preprocess the verse
    verse_tok = word_tokenize(verse.lower().translate(str.maketrans('','',string.punctuation)))
    verse_vectors.append(senemb(1, verse_tok, model, 100))    

In [144]:
# Here we calculate the sentence embedding vectors for queries

# getting the queries
df = pd.read_csv('queries_syn.csv')
queries = df['Verse'].to_list()

# pre-processing the queries
queries = [query.lower().translate(str.maketrans('','',string.punctuation)).split() for query in queries]

# a list to store all the vectors
query_vectors = []

for query in queries:
    query_vectors.append(senemb(1, query, model, 100))

In [145]:
#list1: to store cos. sim. of each query with the verses
list1 = []

for i in query_vectors:
    list2 = []
    for j in verse_vectors:
        d= (dot(i , j))/(norm(i)*norm(j))
        list2.append(d)
    list1.append(list2)

In [146]:
#list3: to store verse nos. of top three verses for each query
#to be put as results in mAP fun.
list3 = []

for i in list1:
    list4 = []
    
    #take three elements with highest scores   
    for ele in sorted(i)[:-4:-1]:
        list4.append(verses.iloc[i.index(ele)][0].replace(":",""))
    list3.append(list4)